In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
'''
@author :wangkechn
https://github.com/wangkechn
'''
#载入数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

In [2]:
#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#z最小值
        tf.summary.histogram('histogram',var)#直方图


#tf.stack()

#命名空间
with tf.name_scope('input'):
    #定义两个placeholder
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    y = tf.placeholder(tf.float32,[None,10],name='y_input')

'''
#创建一个简单的神经网络[784,32,10]
W_L1 = tf.Variable(tf.random_normal([784,32]))
b_L1 = tf.Variable(tf.zeros([32]))
L1 = tf.nn.sigmoid(tf.matmul(x,W_L1)+b_L1)

W_L2 = tf.Variable(tf.random_normal([32,10]))
b_L2 = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(L1,W_L2)+b_L2)

'''

#创建一个简单的神经网络[784,10]
with tf.name_scope('layer'):
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W)
    with tf.name_scope('boases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)

'''
#二次代价函数
loss = tf.reduce_mean(tf.square(y - prediction))
'''
with tf.name_scope('loss'):
    #交叉熵代价函数   该版本的之后会被移除 详见下面的WARNING
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

with tf.name_scope('accurary'):
    with tf.name_scope('correct_prediction'):
        #equal 比较相同 返回True  结果存放在布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  #argmax()返回一维向量中最大的值的索引位置
    with tf.name_scope('accurary'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #cast()转化格式 True为1.0 False:0.0
        tf.summary.scalar('accuracy',accuracy)


#合并所有的summary
merged = tf.summary.merge_all()
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    #重新运行程序 否则图会重叠
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
                       
            
            
            
        writer.add_summary(summary,epoch)    
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter' + str(epoch) + ",Testing Accuracy "+ str(acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter0,Testing Accuracy 0.8242
Iter1,Testing Accuracy 0.8923
Iter2,Testing Accuracy 0.9018
Iter3,Testing Accuracy 0.9054
Iter4,Testing Accuracy 0.9077
Iter5,Testing Accuracy 0.9105
Iter6,Testing Accuracy 0.9113
Iter7,Testing Accuracy 0.9131
Iter8,Testing Accuracy 0.9154
Iter9,Testing Accuracy 0.9159
Iter10,Testing Accuracy 0.9165
Iter11,Testing Accuracy 0.9182
Iter12,Testing Accuracy 0.9189
Iter13,Testing Accuracy 0.9192
Iter14,Testing Accuracy 0.9202
Iter15,Testing Accuracy 0.9207
Iter16,Testing Accuracy 0.9212
Iter17,Testing Accuracy 0.9204
Iter18,Testing Accuracy 0.9213
Iter19,Testing Accuracy 0.9212
Iter20,Testing Accuracy 0.9218


In [ ]:
 for epoch in range(2001):
        #每个批次100个样本
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
            writer.add_summary(summary,i)
            if(i%500==0):
                acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
'''
会有2000个点，2000次的100个样本 训练集运行了多少个周期不定
'''
            